# User routes on the site
## Description
**Clickstream** is a sequence of user actions on a website. It allows you to understand how users interact with the site. In this task, you need to find the most frequent custom routes.

## Input data
Input data is а table with clickstream data in file `hdfs:/data/clickstream.csv`.

### Table structure
* `user_id (int)` - Unique user identifier.
* `session_id (int)` - Unique identifier for the user session. The user's session lasts until the identifier changes.
* `event_type (string)` - Event type from the list:
    * **page** - visit to the page
    * **event** - any action on the page
    * <b>&lt;custom&gt;</b> - string with any other type
* `event_type (string)` - Page on the site.
* `timestamp (int)` - Unix-timestamp of action.

### Browser errors
Errors can sometimes occur in the user's browser - after such an error appears, we can no longer trust the data of this session and all the following lines after the error or at the same time with it are considered corrupted and **should not be counted** in statistics.

When an error occurs on the page, a random string containing the word **error** will be written to the `event_type` field.

### Sample of user session
<pre>
+-------+----------+------------+----------+----------+
|user_id|session_id|  event_type|event_page| timestamp|
+-------+----------+------------+----------+----------+
|    562|       507|        page|      main|1620494781|
|    562|       507|       event|      main|1620494788|
|    562|       507|       event|      main|1620494798|
|    562|       507|        page|    family|1620494820|
|    562|       507|       event|    family|1620494828|
|    562|       507|        page|      main|1620494848|
|    562|       507|wNaxLlerrorU|      main|1620494865|
|    562|       507|       event|      main|1620494873|
|    562|       507|        page|      news|1620494875|
|    562|       507|        page|   tariffs|1620494876|
|    562|       507|       event|   tariffs|1620494884|
|    562|       514|        page|      main|1620728918|
|    562|       514|       event|      main|1620729174|
|    562|       514|        page|   archive|1620729674|
|    562|       514|        page|     bonus|1620729797|
|    562|       514|        page|   tariffs|1620731090|
|    562|       514|       event|   tariffs|1620731187|
+-------+----------+------------+----------+----------+
</pre>

#### Correct user routes for a given user:
* **Session 507**: main-family-main
* **Session 514**: main-archive-bonus-tariffs

Route elements are ordered by the time they appear in the clickstream, from earliest to latest.

The route must be accounted for completely before the end of the session or an error in the session.

## Task
You need to use the Spark SQL, Spark RDD and Spark DF interfaces to create a solution file, the lines of which contain **the 30 most frequent user routes** on the site.

Each line of the file should contain the `route` and `count` values **separated by tabs**, where:
* `route` - route on the site, consisting of pages separated by "-".
* `count` - the number of user sessions in which this route was.

The lines must be **ordered in descending order** of the `count` field.

## Criteria
You can get maximum of 3.5 points (final grade) for this assignment, depedning on the number of interface you manage to leverage. The criteria are as follows:

* 0.5 points – Spark SQL solution with 1 query
* 0.5 points – Spark SQL solution with <=2 queries
* 0.5 points – Spark RDD solution
* 0.5 points – Spark DF solution
* 0.5 points – your solution algorithm is relatively optimized, i.e.: no O^2 or O^3 complexities; appropriate object usage; no data leaks etc. This is evaluated by staff.
* 1 point – 1 on 1 screening session. During this session staff member can ask you questions regarding your solution logic, framework usage, questionable parts of your code etc. If your code is clean enough, the staff member can just ask you to solve a theoretical problem connected to Spark.


In [1]:
! hadoop fs -ls -h /data

Found 2 items
-rw-r--r--   1 root supergroup     30.7 M 2023-09-24 20:38 /data/clickstream.csv
drwxr-xr-x   - root supergroup          0 2023-09-24 20:38 /data/transactions


# Spark SQL solution

In [14]:
import findspark
findspark.init()

import pyspark
sc = pyspark.SparkContext(appName='jupyter')

from pyspark.sql import SparkSession
se = SparkSession(sc)

2023-10-04 22:37:30,151 WARN yarn.Client: Neither spark.yarn.jars nor spark.yarn.archive is set, falling back to uploading libraries under SPARK_HOME.


In [15]:
df = se.read.csv("hdfs:/data/clickstream.csv", header=True, inferSchema=True, sep='\t')

df.createOrReplaceTempView("events")

top_routes = se.sql("""
    WITH
        ErrorFlag AS (
            SELECT *,
                   CASE WHEN event_type LIKE '%error%' THEN 1 ELSE 0 END as is_error
            FROM events
        ),
        MarkedEvents AS (
            SELECT user_id, session_id, event_type, event_page, timestamp,
                   MAX(is_error) OVER (PARTITION BY user_id, session_id ORDER BY user_id, session_id, timestamp) as error_flag
            FROM ErrorFlag
        ),
        FilteredEvents AS (
            SELECT user_id, session_id, event_page, timestamp
            FROM MarkedEvents
            WHERE (event_type = 'page') AND (error_flag = 0)
        ),
        Routes AS (
            SELECT user_id, session_id, CONCAT_WS("-", COLLECT_LIST(event_page)) as route
            FROM FilteredEvents
            GROUP BY user_id, session_id
        )
    SELECT route, COUNT(*) as count
    FROM Routes
    GROUP BY route
    ORDER BY count DESC
    LIMIT 30
""")
top_routes.show()

+--------------------+-----+
|               route|count|
+--------------------+-----+
|                main| 8090|
|        main-archive| 1096|
|         main-rabota| 1039|
|       main-internet|  880|
|          main-bonus|  864|
|           main-news|  760|
|        main-tariffs|  669|
|         main-online|  584|
|          main-vklad|  514|
| main-archive-rabota|  167|
| main-rabota-archive|  167|
|  main-bonus-archive|  139|
|   main-rabota-bonus|  137|
|   main-bonus-rabota|  133|
|    main-news-rabota|  133|
|main-archive-inte...|  131|
|    main-rabota-news|  129|
|main-internet-rabota|  127|
|   main-archive-news|  125|
|main-rabota-internet|  123|
+--------------------+-----+
only showing top 20 rows



In [ ]:
top_routes.write.option("header", True).csv("spark_sql_solution.csv")

! hadoop fs -copyToLocal /user/jovyan/spark_sql_solution.csv

In [ ]:
sc.stop()

# Spark RDD solution
I get different results with RDD than with SQL and DataFrame, but I don't know why. 

In [6]:
import findspark
findspark.init()

import pyspark
sc = pyspark.SparkContext(appName='jupyter')

from pyspark.sql import SparkSession
se = SparkSession(sc)

2023-10-04 22:33:13,352 WARN yarn.Client: Neither spark.yarn.jars nor spark.yarn.archive is set, falling back to uploading libraries under SPARK_HOME.


In [7]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType


df = se.read.csv("hdfs:/data/clickstream.csv", header=True, inferSchema=True, sep='\t')

rdd = df.rdd

sorted_rdd = rdd.sortBy(lambda row: (row.user_id, row.session_id, row.timestamp))

def prepare_routes(iterator):
    route_list = []
    error_detected = False
    current_session = None
    for row in iterator:
        if current_session != row.session_id:
            # output routes from previous session
            if route_list:
                yield '-'.join(route_list), 1

            # Reset for new session
            route_list = []
            error_detected = False
            current_session = row.session_id
        
        if "error" in row.event_type:
            error_detected = True
        
        if row.event_type == 'page' and not error_detected:
            route_list.append(row.event_page)

routes_rdd = sorted_rdd.mapPartitions(prepare_routes)

counted_routes = routes_rdd.reduceByKey(lambda a, b: a + b)
top_routes_rdd = counted_routes.sortBy(lambda x: x[1], ascending=False).take(30)

# Convert RDD back to DataFrame to display
schema = StructType([
    StructField("route", StringType(), True),
    StructField("count", IntegerType(), True)
])
top_routes = se.createDataFrame(top_routes_rdd, schema=schema)
top_routes.show()

+--------------------+-----+
|               route|count|
+--------------------+-----+
|                main| 8091|
|        main-archive| 1096|
|         main-rabota| 1039|
|       main-internet|  879|
|          main-bonus|  865|
|           main-news|  760|
|        main-tariffs|  669|
|         main-online|  584|
|          main-vklad|  514|
| main-archive-rabota|  167|
| main-rabota-archive|  166|
|  main-bonus-archive|  138|
|   main-rabota-bonus|  137|
|    main-news-rabota|  134|
|   main-bonus-rabota|  133|
|main-archive-inte...|  131|
|    main-rabota-news|  129|
|main-internet-rabota|  128|
|   main-archive-news|  125|
|main-internet-arc...|  123|
+--------------------+-----+
only showing top 20 rows



In [8]:
top_routes.repartition(1).write.option("header", True).csv("spark_rdd_solution.csv")

! hadoop fs -copyToLocal /user/jovyan/spark_rdd_solution.csv

In [9]:
sc.stop()

# Spark DF solution

In [10]:
import findspark
findspark.init()

import pyspark
sc = pyspark.SparkContext(appName='jupyter')

from pyspark.sql import SparkSession
se = SparkSession(sc)

2023-10-04 22:35:15,746 WARN yarn.Client: Neither spark.yarn.jars nor spark.yarn.archive is set, falling back to uploading libraries under SPARK_HOME.


In [11]:
from pyspark.sql import functions as F
from pyspark.sql.window import Window

df = se.read.csv("hdfs:/data/clickstream.csv", header=True, inferSchema=True, sep='\t')

windowSpec = Window.partitionBy("user_id", "session_id").orderBy("timestamp")

df = df.withColumn("error_flag", F.max(F.when(F.col("event_type").like("%error%"), 1).otherwise(0)).over(windowSpec))

top_routes = (df.filter((F.col("event_type") == "page") & (F.col("error_flag") == 0))
              .groupBy("user_id", "session_id")
              .agg(F.concat_ws("-", F.collect_list("event_page")).alias("route"))
              .groupBy("route")
              .count()
              .orderBy(F.col("count").desc())
              .limit(30))

top_routes.show()

+--------------------+-----+
|               route|count|
+--------------------+-----+
|                main| 8090|
|        main-archive| 1096|
|         main-rabota| 1039|
|       main-internet|  880|
|          main-bonus|  864|
|           main-news|  760|
|        main-tariffs|  669|
|         main-online|  584|
|          main-vklad|  514|
| main-archive-rabota|  167|
| main-rabota-archive|  167|
|  main-bonus-archive|  139|
|   main-rabota-bonus|  137|
|   main-bonus-rabota|  133|
|    main-news-rabota|  133|
|main-archive-inte...|  131|
|    main-rabota-news|  129|
|main-internet-rabota|  127|
|   main-archive-news|  125|
|main-rabota-internet|  123|
+--------------------+-----+
only showing top 20 rows



In [12]:
top_routes.write.option("header", True).csv("spark_dataframe_solution.csv")

! hadoop fs -copyToLocal /user/jovyan/spark_dataframe_solution.csv

In [13]:
sc.stop()